In [2]:
import sqlite3
import pandas as pd

# establish connection to database
conn = sqlite3.connect( 'brazilian_ecommerce.db')

In [33]:
# create a sample query for understanding dataset

query = """
SELECT DISTINCT pt.product_category_name_english,
    COUNT(DISTINCT o.order_id) as total_orders
FROM orders o
INNER JOIN order_items oi 
ON o.order_id = oi.order_id
INNER JOIN products p 
ON oi.product_id = p.product_id
INNER JOIN product_translations pt
ON p.product_category_name = pt.product_category_name
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

"""

df = pd.read_sql(query, conn)
df.head(50)

,product_category_name_english,total_orders
0,bed_bath_table,9417
1,health_beauty,8836
2,sports_leisure,7720
3,computers_accessories,6689
4,furniture_decor,6449
5,housewares,5884
6,watches_gifts,5624
7,telephony,4199
8,auto,3897
9,toys,3886


In [6]:
# Which products are purchased most frequently?

# query = """
# SELECT DISTINCT p.product_id,
#     pt.product_category_name_english as product_name,
#     COUNT(o.order_id) as total_orders
# FROM orders o
# INNER JOIN order_items oi
# ON o.order_id = oi.order_id
# INNER JOIN products p 
# ON oi.product_id = p.product_id
# INNER JOIN product_translations pt
# ON p.product_category_name = pt.product_category_name
# GROUP BY 1
# ORDER BY 3 DESC
# LIMIT 5


# ;
# """

# df = pd.read_sql(query, conn)
# df.head(100)

# # Done by trying to rank each product by its total number of orders and selecting the top ranked products

# query = """
# WITH total_orders_per_product AS (
#     SELECT product_id,
#         COUNT(order_id) as total_orders
#     FROM order_items
#     GROUP BY 1
# ), product_details AS (
#     SELECT oi.*,
#         RANK() OVER(PARTITION BY t.product_id ORDER BY t.total_orders) as product_frequency_rank
#     FROM total_orders_per_product t
#     LEFT JOIN order_items oi
#     ON t.product_id = oi.product_id
# )
# SELECT pd.product_id,
#     pd.product_frequency_rank
# FROM product_details pd
# WHERE pd.product_frequency_rank < 6

# ;
# """

# df = pd.read_sql(query, conn)
# df.head(100)

query = """
SELECT p.product_id,
    product_category_name_english as product_category,
    COUNT(DISTINCT o.order_purchase_timestamp) as order_frequency
FROM orders o 
INNER JOIN order_items oi
ON o.order_id = oi.order_id
INNER JOIN products p
ON oi.product_id = p.product_id
INNER JOIN product_translations pt 
ON p.product_category_name = pt.product_category_name
GROUP BY 1
ORDER BY 3 DESC
LIMIT 10
;
"""

df = pd.read_sql(query, conn)
df.head(100)

,product_id,product_category,order_frequency
0,99a4788cb24856965c36a24e339b6058,bed_bath_table,467
1,aca2eb7d00ea1a7b8ebd4e68314663af,furniture_decor,431
2,422879e10f46682990de24d770e7f83d,garden_tools,352
3,d1c427060a0f73f6b889a5c7c61f2ac4,computers_accessories,323
4,389d119b48cf3043d311335e499d9c6b,garden_tools,311
5,53b36df67ebb7c41585e8d54d6772e08,watches_gifts,306
6,368c6c730842d78016ad823897a372db,garden_tools,291
7,53759a2ecddad2bb87a079a1f1519f73,garden_tools,287
8,154e7e31ebfa092203795c972e5804a6,health_beauty,269
9,2b4609f8948be18874494203496bc318,health_beauty,259


In [36]:
# Which customers (and what type) are the most frequent / highest value purchasers?

query = """
WITH order_
;
"""

# # customer order frequency
# query = """
# SELECT c.customer_unique_id,
#     COUNT(DISTINCT o.order_id) as order_frequency
# FROM orders o 
# INNER JOIN customers c 
# ON o.customer_id = c.customer_id
# GROUP BY 1
# ORDER BY 2 DESC
# LIMIT 10
# ;
# """

# # order rank by date per customer
# query = """
# SELECT c.customer_unique_id,
#     o.order_id,
#     RANK() OVER(PARTITION BY c.customer_unique_id ORDER BY o.order_purchase_timestamp) as order_date_rank
# FROM orders o 
# INNER JOIN customers c 
# ON o.customer_id = c.customer_id
# GROUP BY 1,2
# ORDER BY 3 DESC
# LIMIT 10
# ;
# """

# # total spent per customer and payment type used
# query = """
# SELECT DISTINCT c.customer_unique_id,
#     op.payment_type,
#     c.customer_city,
#     c.customer_state,
#     o.order_id,
#     SUM(op.payment_value) as total_spent
# FROM customers c 
# INNER JOIN orders o
# ON c.customer_id = o.customer_id
# INNER JOIN order_payments op
# ON o.order_id = op.order_id
# GROUP BY 1
# ORDER BY 6 DESC
# LIMIT 10
# ;
# """

# customer frequency by purchase date
query = """
SELECT c.customer_unique_id,
    COUNT(DISTINCT o.order_purchase_timestamp) as customer_frequency
FROM customers c 
INNER JOIN orders o
ON c.customer_id = o.customer_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
;
"""

df = pd.read_sql(query, conn)
df.head(100)

,customer_unique_id,customer_frequency
0,8d50f5eadf50201ccdcedfb9e2ac8455,17
1,3e43e6105506432c953e165fb2acf44c,9
2,ca77025e7201e3b30c44b472ff346268,7
3,6469f99c1f9dfae7733b25662e7f1782,7
4,1b6c7548a2a1f9037c1fd3ddfed95f33,7
5,f0e310a6839dce9de1638e0fe5ab282a,6
6,de34b16117594161a6a89c50b289d35a,6
7,dc813062e0fc23409cd255f7f53c7074,6
8,63cfc61cee11cbe306bff5857d00bfe4,6
9,12f5d6e1cbf93dafd9dcc19095df0b3d,6


In [3]:
# Which customers (and what type) are the most frequent / highest value purchasers?
query = """
SELECT c.customer_unique_id,
    c.customer_city,
    c.customer_state,
    SUM(op.payment_type)
FROM orders o 
INNER JOIN customers c 
ON o.customer_id = c.customer_id
INNER JOIN order_payments op 
ON o.order_id = op.order_id
GROUP BY 1 
HAVING SUM(op.payment_type) > 2000 
AND COUNT(DISTINCT o.order_purchase_timestamp) > 5
;
"""

df = pd.read_sql(query, conn)
df.head(100)

,customer_unique_id,customer_city,customer_state,SUM(op.payment_type)


In [6]:
# RFM analysis
query = """
WITH customer_calculations AS (
    SELECT DISTINCT c.customer_id,
        MAX(o.order_purchase_timestamp) as last_order_date,
        COUNT(o.order_id) as order_count,
        AVG(op.payment_value) as avg_spent
    FROM orders o 
    INNER JOIN customers c 
    ON o.customer_id = c.customer_id
    INNER JOIN order_payments op 
    ON o.order_id = op.order_id
    GROUP BY 1
), quantiles AS (
    SELECT customer_id,
        NTILE(5) OVER (ORDER BY last_order_date) as rfm_recency,
        NTILE(5) OVER (ORDER BY order_count) as rfm_frequency,
        NTILE(5) OVER (ORDER BY avg_spent) as rfm_monetary
    FROM customer_calculations
), combined_rfm AS (
SELECT customer_id, 
    rfm_recency * 100 + rfm_frequency * 10 + rfm_monetary as rfm_combined
FROM quantiles
)
    SELECT COUNT(DISTINCT customer_id), 
        CASE 
            WHEN rfm_combined > 500 THEN 'Most Active'
            WHEN rfm_combined > 400 AND rfm_combined < 500 THEN 'Active' 
            WHEN rfm_combined > 300 AND rfm_combined < 400 THEN 'Slipping Away'
            WHEN rfm_combined > 200 AND rfm_combined < 300 THEN 'At Risk'
            WHEN rfm_combined > 100 AND rfm_combined < 200 THEN 'Lost'
            ELSE NULL
        END as customer_status
    FROM combined_rfm
    GROUP BY 2
    


    
 

;
"""

df = pd.read_sql(query, conn)
df.head(100)

,COUNT(DISTINCT customer_id),customer_status
0,19888,Active
1,19888,At Risk
2,19888,Lost
3,19888,Most Active
4,19888,Slipping Away


,rfm_combined
0,444
1,443
2,442
3,441
4,434
5,433
6,423
7,422
8,412
9,411


In [10]:
# total spent per customer and payment type used
query = """
SELECT DISTINCT c.customer_unique_id,
    op.payment_type,
    SUM(op.payment_value) as customer_value
FROM order_payments op
INNER JOIN orders o 
ON op.order_id = o.order_id
LEFT JOIN customers c
ON o.customer_id = c.customer_id
GROUP BY 1
ORDER BY 3 DESC
;
"""

df = pd.read_sql(query, conn)
df.head(100)

,customer_unique_id,payment_type,customer_value
0,0a0a92112bd4c708ca5fde585afaa872,credit_card,13664.08
1,46450c74a0d8c5ca9395da1daac6c120,credit_card,9553.02
2,da122df9eeddfedc1dc1f5349a1a690c,credit_card,7571.63
3,763c8b1c9c68a0229c42c9fc6f662b93,boleto,7274.88
4,dc4802a71eae9be1dd28f5d788ceb526,credit_card,6929.31
...,...,...,...
95,8e07cbdbbb24328ece74135e2a81405b,credit_card,2455.12
96,d3ef55d6de8e4be1d4b905e8169224b5,credit_card,2452.12
97,d77aa95864ae5b42160937615628723a,credit_card,2450.10
98,85475e754525d6d222d917dbc8251801,credit_card,2438.51


In [44]:
# What types of metrics should the business be focused on?
query = """
WITH start_pool AS ( 
    SELECT DISTINCT customer_id 
    FROM orders
    WHERE month(purchase_date)=10 
    AND year(purchase_date)=2016)
)
SELECT year(order_purchase_timestamp), 
       month(order_purchase_timestamp), 
       COUNT(DISTINCT customer_id) AS number 
FROM     orders 
WHERE    year(order_purchase_timestamp)=2016 
AND      customer_id IN january_pool 
GROUP BY 1, 
         2
WITH customer_visits AS (
    SELECT   customer_id, 
             datediff(month, '2016-01-01', order_purchase_timestamp) AS purchase_month 
    FROM     orders 
    GROUP BY 1, 
             2 
    ORDER BY 1, 
             2
), time_lapse AS (
    SELECT customer_id,
           purchase_month LEAD(purchase_month, 1) OVER(PARTITION BY customer_id ORDER BY customer_id, purchase_month)
    FROM customer_visits
), time_diff_calculated AS (
    SELECT customer_id,
           purchase_month,
           lead,
           lead - purchase_month AS time_diff
    FROM time_lapse
), customers_categorized AS (
    SELECT customer_id,
           purchase_month,
           CASE
                 WHEN time_diff=1 THEN 'retained',
                 WHEN time_diff>1 THEN 'lagger',
                 WHEN time_diff IS NULL THEN 'lost'
           END AS customer_type
    FROM time_diff_calculated
)
SELECT purchase_month,
       COUNT(b.customer_id) / count(a.customer_id) AS retention
FROM customers_categorized a
LEFT JOIN customers_categorized b 
ON a.purchase_month = b.purchase_month
WHERE b.customer_type = 'retained'
GROUP BY 1


    

;
"""

df = pd.read_sql(query, conn)
df.head(100)
# New buyers (count)
# New buyers by channel / geo / first category they purchased

DatabaseError: Execution failed on sql '
WITH start_pool AS ( 
    SELECT DISTINCT customer_id 
    FROM orders
    WHERE month(purchase_date)=10 
    AND year(purchase_date)=2016)
)
SELECT year(order_purchase_timestamp), 
       month(order_purchase_timestamp), 
       COUNT(DISTINCT customer_id) AS number 
FROM     orders 
WHERE    year(order_purchase_timestamp)=2016 
AND      customer_id IN january_pool 
GROUP BY 1, 
         2
WITH customer_visits AS (
    SELECT   customer_id, 
             datediff(month, '2016-01-01', order_purchase_timestamp) AS purchase_month 
    FROM     orders 
    GROUP BY 1, 
             2 
    ORDER BY 1, 
             2
), time_lapse AS (
    SELECT customer_id,
           purchase_month LEAD(purchase_month, 1) OVER(PARTITION BY customer_id ORDER BY customer_id, purchase_month)
    FROM customer_visits
), time_diff_calculated AS (
    SELECT customer_id,
           purchase_month,
           lead,
           lead - purchase_month AS time_diff
    FROM 

In [ ]:
# New buyer acquisition - are we acquiring buyers consistently / economically?



In [ ]:
# Engagement - are the buyers we acquire having good experiences? (eg reviews)
# Total buyers (count)
# Avg purchases per active customer
# Avg $ spent per active customer
# Avg star rating

In [ ]:
# Retention - are buyers coming back over time? 
# Repurchase rates (what percent of purchasers purchase again within X weeks)
# Cohort retention (group by when they were acquired, and then look at % with purchases X months out into future)
# LTV (sum of purchases per buyer within X days of their first purchase)

In [ ]:
# Has the business seen any notable trends over time?

query = """
SELECT c.customer_unique_id,
     o.order_id
FROM orders o 
INNER JOIN customers c 
ON o.customer_id = c.customer_id
WHERE c.customer_unique_id = '8d50f5eadf50201ccdcedfb9e2ac8455'

;
"""

df = pd.read_sql(query, conn)
df.head(100)